In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [2]:
# input
pvar_f <- '/oak/stanford/groups/mrivas/ukbb24983/exome/pgen/oqfe_2020/ukb24983_exomeOQFE.pvar.zst'
data_d <- '/scratch/groups/mrivas/ukbb24983/exome/annotation/20201025_exome_oqfe_2020'
af_hwe_f   <- file.path(data_d, 'ukb24983_exomeOQFE.afreq_hwe.20201025.pvar.zst')
vep_f      <- file.path(data_d, 'UKBexomeOQFE.vep101.tsv.gz')
liftOver_f <- file.path(data_d, 'UKBexomeOQFE.hg19.tsv.gz')
vep_csq_f  <- file.path('..', 'VEP_consequence_group.tsv')

# output
annot_f         <- file.path(data_d, 'ukb24983_exomeOQFE.annotation.tsv')
annot_compact_f <- file.path(data_d, 'ukb24983_exomeOQFE.annotation.compact.tsv')



In [3]:
compact_fields <- c(
    'CHROM', 'POS', 'ID', 'REF', 'ALT', 'FILTER',
    'Allele', 'Csq', 'Consequence', 'SYMBOL', 'Gene',
    'f_miss', 'UKB_white_british_hwe_p', 'UKB_white_british_AF', 'UKB_AF',
    'CHROM_hg19', 'POS_hg19', 'REF_hg19', 'ALT_hg19', 'liftOver_unmapped_reason'
)


In [33]:
# functions

cat_or_zcat <- function(f){
    ifelse(endsWith(f, '.zst'), 'zstdcat', ifelse(endsWith(f, '.gz'), 'zcat', 'cat'))
}

fread_CHROM <- function(f, select=NULL){
    fread(cmd=paste(cat_or_zcat(f), f, "| sed -e 's/^chr//g'",  ' | head -n10000'), colClasses = c('#CHROM'='character'), select=select) %>% rename('CHROM'='#CHROM')
}


In [34]:
pvar_f     %>% fread_CHROM() -> pvar_df
liftOver_f %>% fread_CHROM() -> liftOver_df


In [35]:
af_hwe_f %>% fread_CHROM() -> af_hwe_df
vep_f    %>% fread_CHROM() -> vep_df


In [36]:
vep_csq_f %>% fread(select=c('#Consequence', 'Csq')) %>% rename('Consequence'='#Consequence') -> vep_csq_df


In [37]:
pvar_df %>%
left_join(vep_df, by=c('CHROM', 'POS', 'REF', 'ALT')) %>%
left_join(vep_csq_df, by='Consequence') %>%
left_join(af_hwe_df, by=c('CHROM', 'POS', 'ID', 'REF', 'ALT')) %>%
left_join(liftOver_df, by=c('CHROM', 'POS', 'ID', 'REF', 'ALT')) -> full_df


In [34]:
full_df %>%
select(all_of(c(compact_fields, setdiff(colnames(full_df), compact_fields)))) %>%
rename('#CHROM' = 'CHROM') %>%
fwrite(annot_f, sep='\t', na = "NA", quote=F)


In [35]:
full_df %>%
select(all_of(compact_fields)) %>%
rename('#CHROM' = 'CHROM') %>%
fwrite(annot_compact_f, sep='\t', na = "NA", quote=F)


In [33]:
dim(full_df)

[1]   9 154

In [7]:
colnames(pvar_df)

[1] "CHROM" "POS"   "ID"    "REF"   "ALT"

In [8]:
colnames(liftOver_df)

[1] "CHROM"                    "POS"                     
 [3] "ID"                       "REF"                     
 [5] "ALT"                      "CHROM_hg19"              
 [7] "POS_hg19"                 "REF_hg19"                
 [9] "ALT_hg19"                 "liftOver_unmapped_reason"

In [9]:
colnames(af_hwe_df)

[1] "CHROM"                                 
 [2] "POS"                                   
 [3] "ID"                                    
 [4] "REF"                                   
 [5] "ALT"                                   
 [6] "UKB_AF"                                
 [7] "UKB_white_british_AF"                  
 [8] "UKB_non_british_white_AF"              
 [9] "UKB_african_AF"                        
[10] "UKB_s_asian_AF"                        
[11] "UKB_e_asian_AF"                        
[12] "UKB_related_AF"                        
[13] "UKB_others_AF"                         
[14] "UKB_hwe_p"                             
[15] "UKB_white_british_hwe_p"               
[16] "UKB_non_british_white_hwe_p"           
[17] "UKB_african_hwe_p"                     
[18] "UKB_s_asian_hwe_p"                     
[19] "UKB_e_asian_hwe_p"                     
[20] "UKB_related_hwe_p"                     
[21] "UKB_others_hwe_p"                      
[22] "UKB_OBS_CT"                            
[23] "UKB_white_british_OBS_CT"              
[24] "UKB_non_british_white_OBS_CT"          
[25] "UKB_african_OBS_CT"                    
[26] "UKB_s_asian_OBS_CT"                    
[27] "UKB_e_asian_OBS_CT"                    
[28] "UKB_related_OBS_CT"                    
[29] "UKB_others_OBS_CT"                     
[30] "UKB_MISSING_CT"                        
[31] "UKB_white_british_MISSING_CT"          
[32] "UKB_non_british_white_MISSING_CT"      
[33] "UKB_african_MISSING_CT"                
[34] "UKB_s_asian_MISSING_CT"                
[35] "UKB_e_asian_MISSING_CT"                
[36] "UKB_related_MISSING_CT"                
[37] "UKB_others_MISSING_CT"                 
[38] "UKB_HOM_REF_CT"                        
[39] "UKB_white_british_HOM_REF_CT"          
[40] "UKB_non_british_white_HOM_REF_CT"      
[41] "UKB_african_HOM_REF_CT"                
[42] "UKB_s_asian_HOM_REF_CT"                
[43] "UKB_e_asian_HOM_REF_CT"                
[44] "UKB_related_HOM_REF_CT"                
[45] "UKB_others_HOM_REF_CT"                 
[46] "UKB_HET_REF_ALT_CTS"                   
[47] "UKB_white_british_HET_REF_ALT_CTS"     
[48] "UKB_non_british_white_HET_REF_ALT_CTS" 
[49] "UKB_african_HET_REF_ALT_CTS"           
[50] "UKB_s_asian_HET_REF_ALT_CTS"           
[51] "UKB_e_asian_HET_REF_ALT_CTS"           
[52] "UKB_related_HET_REF_ALT_CTS"           
[53] "UKB_others_HET_REF_ALT_CTS"            
[54] "UKB_TWO_ALT_GENO_CTS"                  
[55] "UKB_white_british_TWO_ALT_GENO_CTS"    
[56] "UKB_non_british_white_TWO_ALT_GENO_CTS"
[57] "UKB_african_TWO_ALT_GENO_CTS"          
[58] "UKB_s_asian_TWO_ALT_GENO_CTS"          
[59] "UKB_e_asian_TWO_ALT_GENO_CTS"          
[60] "UKB_related_TWO_ALT_GENO_CTS"          
[61] "UKB_others_TWO_ALT_GENO_CTS"           
[62] "UKB_HAP_REF_CT"                        
[63] "UKB_white_british_HAP_REF_CT"          
[64] "UKB_non_british_white_HAP_REF_CT"      
[65] "UKB_african_HAP_REF_CT"                
[66] "UKB_s_asian_HAP_REF_CT"                
[67] "UKB_e_asian_HAP_REF_CT"                
[68] "UKB_related_HAP_REF_CT"                
[69] "UKB_others_HAP_REF_CT"                 
[70] "UKB_HAP_ALT_CTS"                       
[71] "UKB_white_british_HAP_ALT_CTS"         
[72] "UKB_non_british_white_HAP_ALT_CTS"     
[73] "UKB_african_HAP_ALT_CTS"               
[74] "UKB_s_asian_HAP_ALT_CTS"               
[75] "UKB_e_asian_HAP_ALT_CTS"               
[76] "UKB_related_HAP_ALT_CTS"               
[77] "UKB_others_HAP_ALT_CTS"                
[78] "f_miss"

In [10]:
colnames(vep_df)

[1] "CHROM"                 "POS"                   "REF"                  
 [4] "ALT"                   "FILTER"                "Allele"               
 [7] "Consequence"           "IMPACT"                "SYMBOL"               
[10] "Gene"                  "Feature_type"          "Feature"              
[13] "BIOTYPE"               "EXON"                  "INTRON"               
[16] "HGVSc"                 "HGVSp"                 "cDNA_position"        
[19] "CDS_position"          "Protein_position"      "Amino_acids"          
[22] "Codons"                "Existing_variation"    "ALLELE_NUM"           
[25] "DISTANCE"              "STRAND"                "FLAGS"                
[28] "VARIANT_CLASS"         "SYMBOL_SOURCE"         "HGNC_ID"              
[31] "CANONICAL"             "MANE"                  "TSL"                  
[34] "APPRIS"                "CCDS"                  "ENSP"                 
[37] "SWISSPROT"             "TREMBL"                "UNIPARC"              
[40] "GENE_PHENO"            "SIFT"                  "PolyPhen"             
[43] "DOMAINS"               "miRNA"                 "HGVS_OFFSET"          
[46] "AF"                    "AFR_AF"                "AMR_AF"               
[49] "EAS_AF"                "EUR_AF"                "SAS_AF"               
[52] "AA_AF"                 "EA_AF"                 "gnomAD_AF"            
[55] "gnomAD_AFR_AF"         "gnomAD_AMR_AF"         "gnomAD_ASJ_AF"        
[58] "gnomAD_EAS_AF"         "gnomAD_FIN_AF"         "gnomAD_NFE_AF"        
[61] "gnomAD_OTH_AF"         "gnomAD_SAS_AF"         "MAX_AF"               
[64] "MAX_AF_POPS"           "CLIN_SIG"              "SOMATIC"              
[67] "PHENO"                 "PUBMED"                "VAR_SYNONYMS"         
[70] "MOTIF_NAME"            "MOTIF_POS"             "HIGH_INF_POS"         
[73] "MOTIF_SCORE_CHANGE"    "TRANSCRIPTION_FACTORS"